In [ ]:
from bs4 import BeautifulSoup
import requests
import sqlite3

TIKI_URL = 'https://tiki.vn'

In [ ]:
conn = sqlite3.connect('tiki.db')
cur = conn.cursor()

In [ ]:
cur.execute('DROP TABLE categories;')
conn.commit()

In [ ]:
# Get the HTML content get_url()
def get_url(url):
    try:
        response = requests.get(url).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)
    except 

In [ ]:
# Create table categories in the database using a function
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER,
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query) # execute the query
        conn.commit() # make changes
    except Exception as err: 
        print('ERROR BY CREATE TABLE', err)
        
create_categories_table()

In [ ]:
cur.execute('SELECT * FROM categories').fetchall() # there you go, table was created, as good as new

In [ ]:
# time to add cats to the table, but wait, let's create some cool OOP atributes and methods first

class Category:
    
    # create some atributes
    def __init__(self, name, url, parent_id = None,have_child = 1, cat_id= None):
        self.name = name
        self.url = url
        self.parent_id = parent_id
        self.cat_id = cat_id
        self.have_child = have_child
    
    # how to display this cat when being called
    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id} "
    
    # so many things are going on here, let's dive in
    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid # this creates the cat_id, it assigns that cat_id atribute to lastrowid, so it changes the repr, woohoo
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)

In [ ]:
# let's scrape the main cats:
def get_main_cats(save_db = False):
    
    # get our soup from the pre-defined get_url function
    soup = get_url(TIKI_URL)
    
    main_cats = []
    
    for a in soup.find_all('a',class_="MenuItem__MenuLink-sc-181aa19-1 fKvTQu"):
        name = a.find('span', class_='text').text
        url = a['href']
        cat = Category(name, url)
        main_cats.append(cat)
        
        
        if save_db:
            cat.save_into_db()
        
    return main_cats
        
    
main_cats = get_main_cats(save_db = True) 
main_cats

In [ ]:
# re is our good cleaner to tidy things up
import re


# get_sub_categories() given a parent category
def get_sub_categories(parent_category, save_db = False):
    parent_url = parent_category.url
    result = []
    
    try:
        soup = get_url(parent_url)
        for div in soup.find_all('div',class_='list-group-item is-child'):
            name = div.a.text
        
            # here comes our cleaner
            name = re.sub('\s+|\\n', ' ', name)
            
            sub_url = TIKI_URL + div.a['href']
            
            cat = Category(name, sub_url, parent_category.cat_id)
            result.append(cat)
            
            if save_db:
                cat.save_into_db()
            
            
        
    except Exception as err:
        print('ERROR BY GET SUB CATEGORIES:', err)

    return result

In [ ]:
x = main_cats[0]
x

In [ ]:
# get_all_categories() given a list of main categories (This is a recusion function)
def get_all_categories(categories):
    if len(categories) == 0:
        return
    
    for cat  in categories:
        sub_cat = get_sub_categories(cat, save_db = True)
        print (sub_cat)
        get_all_categories(sub_cat)

In [ ]:
get_all_categories(main_cats)

In [ ]:
cur.execute('SELECT * FROM categories').fetchall()

In [ ]:
import pandas as pd
cat_table = pd.read_sql_query('SELECT * FROM categories', conn)
cat_table

In [ ]:
Final_layer = pd.read_sql_query('SELECT table1.id, table1.name, table1.url FROM categories AS table1 LEFT JOIN categories AS table2 ON table1.id = table2.parent_id WHERE table2.parent_id IS NULL',conn)

In [ ]:
Final_layer.to_csv("./links.csv", index = False)

In [ ]:
# Scrape all the products from these above links 

In [ ]:
# get the list of all the links
links = list(Final_layer['url'])
links


In [ ]:
def scrape_tiki_all(url = links):
    data = []
    for url in links:

        url_base = url
        page = 1

        items = True
        while items != [] and page <2:
            try:
                
                soup = get_url(url)        
                items = soup.find_all('div',{"class": "product-item"}) 

                for item in items: 
                    try: 

                        # Each tag is dictionary containing the required information
                        dic = {"product_id":"","category":"","seller_id":"","title":"","price":"","image_url":""}

                        dic['category'] = item['data-category'].split("/")[0]
                        dic["product_id"] = item["data-id"]
                        dic["seller_id"] = item["data-seller-product-id"]
                        dic["title"] = item["data-title"]
                        dic["price"] = item["data-price"]

                        # There are some items without img tag...
                        if item.find("span",{"class":"image"}):
                            dic["image_url"] = item.find("span",{"class":"image"}).img["src"]

                        # Append the dictionary to data list
                        data.append(dic)
                    except:

                         # Skip if error and print error message
                        print("We got an error")
                # print out the page number and items to keep track
                print(page, len(data))
            
            except:
                print("er")
            # increment page
            page += 1

            # create the url of the next page
            url = url_base + "&page=" +str(page)


    return data

In [ ]:
all_products = scrape_tiki_all(url = links)

In [ ]:
all_products

In [ ]:
# Create table categories in the database using a function
def create_products_table():
    query = """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_ID VARCHAR(255),
            Seller_ID VARCHAR(255),
            name VARCHAR(255),
            price VARCHAR(255),
            category TEXT,
            image_url TEXT, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query) # execute the query
        conn.commit() # make changes
    except Exception as err: 
        print('ERROR BY CREATE TABLE', err)
        
create_products_table()


In [ ]:
cur.execute('DROP TABLE categories;')
conn.commit()

In [ ]:
cur.execute('SELECT * FROM products').fetchall()

In [ ]:
p1 = Product("1233","ABS","Huah","819197","Trang Suc","link1") #testing
cur.execute('SELECT * FROM products').fetchall()

In [ ]:
def scrape_tiki_all(url = links, save_db = False):
    data = []
    for url in links:

        url_base = url
        page = 1

        items = True
        while items != [] and page <2:
            try:
                
                soup = get_url(url)        
                items = soup.find_all('div',{"class": "product-item"}) 

                for item in items: 
                    try: 

                        # Each tag is dictionary containing the required information
                        

                        category = item['data-category'].split("/")[0]
                        product_id = item["data-id"]
                        seller_id = item["data-seller-product-id"]
                        name = item["data-title"]
                        price = item["data-price"]
                        

                        # There are some items without img tag...
                        if item.find("span",{"class":"image"}):
                            img = item.find("span",{"class":"image"}).img["src"]
                        product = Product(product_id,seller_id,name,price,category,img)

                        # Append the dictionary to data list
                        data.append(product)
                        
                        if save_db:
                            product.save_into_db()
                    except:

                         # Skip if error and print error message
                        print("We got an error")
                # print out the page number and items to keep track
                print(len(data))
            
            except:
                print("IncompleteRead")
            # increment page
            page += 1

            # create the url of the next page
            url = url_base + "&page=" +str(page)

    return data

In [ ]:
scrape_tiki_all(save_db = True)

In [ ]:
product_table = pd.read_sql_query('SELECT * FROM products', conn)
product_table